In [0]:
%sql
CREATE OR REPLACE TABLE geospatial.greenspaces.entry_ranks_gold AS ( 
SELECT 
ga.id,
ga.ref_to_greenspace_site AS park_id,
gs.park_rank,
ga.access_type,
rn.degree,
ga.distance_to_road_node,
row_number() OVER (PARTITION BY gs.id ORDER BY rn.degree DESC, ga.distance_to_road_node ASC) AS entry_rank,
ga.geometry
FROM geospatial.greenspaces.top_parks_gold gs
LEFT JOIN geospatial.greenspaces.access_point_silver ga
ON gs.id = ga.ref_to_greenspace_site
LEFT JOIN geospatial.networks.road_node_silver rn
ON ga.nearest_road_node_fid = rn.fid
);

In [0]:
%sql
CREATE OR REPLACE TABLE geospatial.greenspaces.entry_rank1_gold AS (
WITH A AS (
SELECT 
ga.id,
ga.ref_to_greenspace_site AS park_id,
gs.park_rank,
ga.access_type,
rn.degree,
ga.distance_to_road_node,
row_number() OVER (PARTITION BY gs.id ORDER BY rn.degree DESC, ga.distance_to_road_node ASC) AS entry_rank,
ga.geometry,
ST_X(ST_Transform(ST_GeomFromEWKB(ga.geometry),'epsg:27700','epsg:4326')) AS longitude,
ST_Y(ST_Transform(ST_GeomFromEWKB(ga.geometry),'epsg:27700','epsg:4326')) AS latitude
FROM geospatial.greenspaces.top_parks_gold gs
LEFT JOIN geospatial.greenspaces.access_point_silver ga
ON gs.id = ga.ref_to_greenspace_site
LEFT JOIN geospatial.networks.road_node_silver rn
ON ga.nearest_road_node_fid = rn.fid) 
SELECT * FROM A WHERE entry_rank = 1
);